# IndexData 数据查看器

本notebook用于展示IndexData类中所有fetcher获取的实际数据内容，帮助快速了解每个指数相关接口的功能和数据结构。

## 📊 IndexData包含的数据类型

- **指数成分股数据** - 指定指数包含的所有成分股信息
- **指数列表数据** - A股市场所有可用指数的列表
- **美股指数数据** - 美国股市主要指数的实时行情

## 1. 导入库并初始化

In [1]:
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 获取项目根目录
current_dir = os.path.abspath('.')
project_root = os.path.dirname(current_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 导入IndexData
from china_stock_data import IndexData

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 10)

print("✅ 导入成功")
print(f"项目根目录: {project_root}")

✅ 导入成功
项目根目录: /Users/juzhongsun/Codes/projects/china_stock_data


## 2. 创建IndexData实例

In [2]:
# 创建IndexData实例 - 以沪深300指数为例
index_data = IndexData(index='000300')

print(f"IndexData实例创建成功！")
print(f"指数代码: {index_data.index}")
print(f"实例Key: {index_data.key()}")
print(f"可用fetchers: {len(index_data.fetchers)}个")

# 获取所有fetcher信息
unique_fetchers = {}
for name, fetcher in index_data.fetchers.items():
    fetcher_id = id(fetcher)
    if fetcher_id not in unique_fetchers:
        unique_fetchers[fetcher_id] = {
            'instance': fetcher,
            'names': [name],
            'class_name': fetcher.__class__.__name__
        }
    else:
        unique_fetchers[fetcher_id]['names'].append(name)

print(f"唯一fetchers: {len(unique_fetchers)}个")
print(f"访问名称总数: {len(index_data.fetchers)}个")

IndexData实例创建成功！
指数代码: 000300
实例Key: I000300
可用fetchers: 6个
唯一fetchers: 3个
访问名称总数: 6个


## 3. 查看所有可用的Fetchers

In [3]:
print("📋 所有可用的Index Data Fetchers:")
print("=" * 60)

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    print(f"{i}. {class_name}")
    print(f"   访问名称: {all_names}")
    print()

print(f"📊 访问名称详细列表:")
fetcher_names = sorted(index_data.fetchers.keys())
for i, name in enumerate(fetcher_names, 1):
    print(f"{i:2d}. {name}")

📋 所有可用的Index Data Fetchers:
1. IndexComponentsFetcher
   访问名称: index_components | index_stock_cons_csindex

2. IndexListFetcher
   访问名称: index_list | index_stock_info

3. UsIndexFetcher
   访问名称: us_index | index_us_stock_sina

📊 访问名称详细列表:
 1. index_components
 2. index_list
 3. index_stock_cons_csindex
 4. index_stock_info
 5. index_us_stock_sina
 6. us_index


## 4. 数据展示函数

In [4]:
def display_index_fetcher_data(fetcher_name, description, index_instance):
    """
    展示指定fetcher的数据内容和结构
    """
    print(f"\n{'='*80}")
    print(f"📊 {description}")
    print(f"🔗 访问名称: {fetcher_name}")
    print(f"📍 当前指数: {index_instance.index or 'N/A'}")
    print(f"{'='*80}")
    
    try:
        # 获取数据
        data = index_instance.get_data(fetcher_name)
        
        if data.empty:
            print("❌ 数据为空")
            return
        
        # 基本信息
        print(f"📈 数据形状: {data.shape[0]}行 x {data.shape[1]}列")
        print(f"📋 列名: {list(data.columns)}")
        
        # 数据类型
        print(f"\n🔢 数据类型:")
        for col, dtype in data.dtypes.items():
            print(f"   {col}: {dtype}")
        
        # 数据预览
        print(f"\n📄 数据预览 (前5行):")
        print(data.head())
        
        if len(data) > 10:
            print(f"\n📄 数据预览 (后5行):")
            print(data.tail())
        
        # 数值列的统计信息
        numeric_cols = data.select_dtypes(include=['number']).columns
        if len(numeric_cols) > 0:
            print(f"\n📊 数值列统计:")
            print(data[numeric_cols].describe())
        
        # 特殊信息提取
        if 'weight' in [col.lower() for col in data.columns]:
            weight_col = [col for col in data.columns if 'weight' in col.lower()][0]
            total_weight = data[weight_col].sum()
            print(f"\n⚖️ 权重信息: 总权重 = {total_weight:.2f}%")
        
        print(f"\n✅ 数据获取成功")
        
    except Exception as e:
        print(f"❌ 获取数据时出错: {str(e)}")

## 5. 指数成分股数据

获取指定指数（如沪深300）的所有成分股信息，包括股票代码、名称、权重等。

In [5]:
display_index_fetcher_data('index_components', '指数成分股数据 - 获取指数包含的所有成分股及其权重信息', index_data)


📊 指数成分股数据 - 获取指数包含的所有成分股及其权重信息
🔗 访问名称: index_components
📍 当前指数: 000300
featch index components 000300
📈 数据形状: 300行 x 9列
📋 列名: ['日期', '指数代码', '指数名称', '指数英文名称', '成分券代码', '成分券名称', '成分券英文名称', '交易所', '交易所英文名称']

🔢 数据类型:
   日期: object
   指数代码: object
   指数名称: object
   指数英文名称: object
   成分券代码: object
   成分券名称: object
   成分券英文名称: object
   交易所: object
   交易所英文名称: object

📄 数据预览 (前5行):
           日期    指数代码   指数名称   指数英文名称   成分券代码  成分券名称  \
0  2025-09-26  000300  沪深300  CSI 300  000001   平安银行   
1  2025-09-26  000300  沪深300  CSI 300  000002    万科A   
2  2025-09-26  000300  沪深300  CSI 300  000063   中兴通讯   
3  2025-09-26  000300  沪深300  CSI 300  000100  TCL科技   
4  2025-09-26  000300  沪深300  CSI 300  000157   中联重科   

                         成分券英文名称      交易所                  交易所英文名称  
0         Ping An Bank Co., Ltd.  深圳证券交易所  Shenzhen Stock Exchange  
1             China Vanke Co Ltd  深圳证券交易所  Shenzhen Stock Exchange  
2                ZTE Corporation  深圳证券交易所  Shenzhen Stock Exchange  
3  TC

## 6. 指数列表数据

获取A股市场所有可用指数的列表信息。

In [6]:
display_index_fetcher_data('index_list', 'A股指数列表数据 - 获取市场上所有可用指数的基本信息', index_data)


📊 A股指数列表数据 - 获取市场上所有可用指数的基本信息
🔗 访问名称: index_list
📍 当前指数: 000300
Fetching index list data!
📈 数据形状: 726行 x 3列
📋 列名: ['index_code', 'display_name', 'publish_date']

🔢 数据类型:
   index_code: object
   display_name: object
   publish_date: object

📄 数据预览 (前5行):
  index_code display_name publish_date
0     000001         上证指数   1991-07-15
1     000002         A股指数   1992-02-21
2     000003         B股指数   1992-02-21
3     000004         工业指数   1993-05-03
4     000005         商业指数   1993-05-03

📄 数据预览 (后5行):
    index_code display_name publish_date
721     399994   中证信息安全主题指数   2015-03-12
722     399995     中证基建工程指数   2015-03-12
723     399996     中证智能家居指数   2014-09-17
724     399997       中证白酒指数   2015-01-21
725     399998       中证煤炭指数   2015-02-13

✅ 数据获取成功


## 7. 美股指数数据

获取美国股市主要指数的实时行情数据。

In [7]:
display_index_fetcher_data('us_index', '美股指数数据 - 获取美国股市主要指数的实时行情', index_data)


📊 美股指数数据 - 获取美国股市主要指数的实时行情
🔗 访问名称: us_index
📍 当前指数: 000300
Fetching us index data!: INX
📈 数据形状: 5472行 x 7列
📋 列名: ['date', 'open', 'high', 'low', 'close', 'volume', 'amount']

🔢 数据类型:
   date: object
   open: float64
   high: float64
   low: float64
   close: float64
   volume: int64
   amount: int64

📄 数据预览 (前5行):
         date     open     high        low    close      volume         amount
0  2004-01-02  1111.92  1118.85  1105.0800  1108.48  1153200000  1281300000000
1  2004-01-05  1108.48  1122.22  1108.4800  1122.22  1578200064  1760250000000
2  2004-01-06  1122.22  1124.46  1118.4399  1123.67  1494499968  1677120000000
3  2004-01-07  1123.67  1126.33  1116.4500  1126.33  1704899968  1914940000000
4  2004-01-08  1126.33  1131.92  1124.9100  1131.92  1868400000  2108990000000

📄 数据预览 (后5行):
            date       open       high        low      close      volume  \
5467  2025-09-22  6654.2798  6698.8799  6648.0698  6693.7500  3275038698   
5468  2025-09-23  6692.4399  6699.5200  66

## 8. 多名称访问演示

演示同一个fetcher可以通过不同名称访问的功能。

In [8]:
print("🔄 多名称访问模式演示")
print("同一个fetcher可以通过业务名称和akshare方法名访问")
print("="*60)

# 展示多名称访问的例子
multi_name_examples = [
    ('index_components', 'index_stock_cons', '指数成分股'),
    ('index_list', 'index_stock_info_sina', '指数列表'),
    ('us_index', 'index_us_stock_sina', '美股指数')
]

for business_name, akshare_name, description in multi_name_examples:
    print(f"\n📋 {description}:")
    
    # 验证两个名称是否指向同一个fetcher实例
    if business_name in index_data.fetchers and akshare_name in index_data.fetchers:
        same_instance = index_data.fetchers[business_name] is index_data.fetchers[akshare_name]
        print(f"   业务名称: {business_name}")
        print(f"   akshare名称: {akshare_name}")
        print(f"   同一实例: {'✅ 是' if same_instance else '❌ 否'}")
        
        # 展示数据形状
        try:
            data = index_data.get_data(business_name)
            print(f"   数据形状: {data.shape if not data.empty else '空数据'}")
        except:
            print(f"   数据形状: 获取失败")
    else:
        available_names = [name for name in [business_name, akshare_name] if name in index_data.fetchers]
        missing_names = [name for name in [business_name, akshare_name] if name not in index_data.fetchers]
        print(f"   ✅ 可用名称: {available_names}")
        print(f"   ❌ 缺失名称: {missing_names}")

print(f"\n📊 总计访问名称: {len(index_data.fetchers)}个")
print(f"📊 唯一fetchers: {len(unique_fetchers)}个")

🔄 多名称访问模式演示
同一个fetcher可以通过业务名称和akshare方法名访问

📋 指数成分股:
   ✅ 可用名称: ['index_components']
   ❌ 缺失名称: ['index_stock_cons']

📋 指数列表:
   ✅ 可用名称: ['index_list']
   ❌ 缺失名称: ['index_stock_info_sina']

📋 美股指数:
   业务名称: us_index
   akshare名称: index_us_stock_sina
   同一实例: ✅ 是
   数据形状: (5472, 7)

📊 总计访问名称: 6个
📊 唯一fetchers: 3个


## 9. 不同指数的数据对比

创建不同指数的IndexData实例，对比它们的成分股数据。

In [9]:
print("📊 不同指数数据对比")
print("="*60)

# 创建不同指数的实例
indices_to_compare = [
    ('000300', '沪深300'),
    ('000905', '中证500'),
    ('399006', '创业板指')
]

index_comparison = {}

for index_code, index_name in indices_to_compare:
    print(f"\n🔍 {index_name} ({index_code}):")
    
    try:
        # 创建指数实例
        idx = IndexData(index=index_code)
        print(f"   实例Key: {idx.key()}")
        
        # 获取成分股数据
        components = idx.get_data('index_components')
        if not components.empty:
            print(f"   成分股数量: {len(components)}只")
            print(f"   数据列数: {components.shape[1]}列")
            print(f"   列名: {list(components.columns)[:3]}..." if len(components.columns) > 3 else f"   列名: {list(components.columns)}")
            
            # 保存用于后续分析
            index_comparison[index_name] = {
                'code': index_code,
                'components_count': len(components),
                'data': components
            }
        else:
            print(f"   ❌ 成分股数据为空")
            
    except Exception as e:
        print(f"   ❌ 错误: {str(e)[:50]}...")

# 对比总结
if index_comparison:
    print(f"\n📈 指数对比总结:")
    print("-" * 40)
    for name, info in index_comparison.items():
        print(f"   {name} ({info['code']}): {info['components_count']}只成分股")
    
    # 找出成分股数量最多和最少的指数
    max_components = max(index_comparison.values(), key=lambda x: x['components_count'])
    min_components = min(index_comparison.values(), key=lambda x: x['components_count'])
    
    max_name = [name for name, info in index_comparison.items() if info['components_count'] == max_components['components_count']][0]
    min_name = [name for name, info in index_comparison.items() if info['components_count'] == min_components['components_count']][0]
    
    print(f"\n🏆 成分股最多: {max_name} ({max_components['components_count']}只)")
    print(f"🏅 成分股最少: {min_name} ({min_components['components_count']}只)")

📊 不同指数数据对比

🔍 沪深300 (000300):
   实例Key: I000300
   成分股数量: 300只
   数据列数: 9列
   列名: ['日期', '指数代码', '指数名称']...

🔍 中证500 (000905):
   实例Key: I000905
featch index components 000905


   成分股数量: 500只
   数据列数: 9列
   列名: ['日期', '指数代码', '指数名称']...

🔍 创业板指 (399006):
   实例Key: I399006
featch index components 399006
Error fetching data: Excel file format cannot be determined, you must specify an engine manually.
   ❌ 错误: '成分券代码'...

📈 指数对比总结:
----------------------------------------
   沪深300 (000300): 300只成分股
   中证500 (000905): 500只成分股

🏆 成分股最多: 中证500 (500只)
🏅 成分股最少: 沪深300 (300只)


## 10. 完整总览

In [10]:
print("📊 IndexData完整功能总览")
print("="*80)

print(f"\n🏗️ 系统信息:")
print(f"   指数代码: {index_data.index}")
print(f"   实例Key: {index_data.key()}")
print(f"   符号参数: {index_data.symbol}")

print(f"\n📈 数据获取能力:")
working_count = 0
total_data_rows = 0

for i, (fetcher_id, info) in enumerate(unique_fetchers.items(), 1):
    primary_name = info['names'][0]
    class_name = info['class_name']
    all_names = ' | '.join(info['names'])
    
    try:
        data = index_data.get_data(primary_name)
        if not data.empty:
            working_count += 1
            total_data_rows += len(data)
            status = f"✅ {data.shape[0]}行 x {data.shape[1]}列"
        else:
            status = "⚠️ 空数据"
    except Exception as e:
        status = f"❌ 错误"
    
    print(f"   {i}. {class_name}: {status}")
    print(f"      访问名称: {all_names}")

print(f"\n🎯 统计结果:")
print(f"   正常工作的fetchers: {working_count}/{len(unique_fetchers)}")
print(f"   总数据行数: {total_data_rows:,}行")
print(f"   访问名称总数: {len(index_data.fetchers)}个")
print(f"   成功率: {working_count/len(unique_fetchers)*100:.1f}%")

print(f"\n💡 使用建议:")
print(f"   • 使用业务名称访问更直观: index.index_components, index.index_list")
print(f"   • 使用akshare名称保持兼容性: index.index_stock_cons")
print(f"   • 通过get_data()方法统一访问: index.get_data('index_components')")
print(f"   • 创建不同指数实例获取对应数据: IndexData(index='000300')")
print(f"   • 所有数据都支持pandas DataFrame操作")

print(f"\n🔍 主要应用场景:")
print(f"   • 指数成分股分析 - 了解指数构成和权重分布")
print(f"   • 指数比较研究 - 对比不同指数的特征")
print(f"   • 投资组合构建 - 基于指数成分股构建投资策略")
print(f"   • 美股指数监控 - 跟踪美国市场主要指数表现")

📊 IndexData完整功能总览

🏗️ 系统信息:
   指数代码: 000300
   实例Key: I000300
   符号参数: None

📈 数据获取能力:
   1. IndexComponentsFetcher: ✅ 300行 x 9列
      访问名称: index_components | index_stock_cons_csindex
   2. IndexListFetcher: ✅ 726行 x 3列
      访问名称: index_list | index_stock_info
   3. UsIndexFetcher: ✅ 5472行 x 7列
      访问名称: us_index | index_us_stock_sina

🎯 统计结果:
   正常工作的fetchers: 3/3
   总数据行数: 6,498行
   访问名称总数: 6个
   成功率: 100.0%

💡 使用建议:
   • 使用业务名称访问更直观: index.index_components, index.index_list
   • 使用akshare名称保持兼容性: index.index_stock_cons
   • 通过get_data()方法统一访问: index.get_data('index_components')
   • 创建不同指数实例获取对应数据: IndexData(index='000300')
   • 所有数据都支持pandas DataFrame操作

🔍 主要应用场景:
   • 指数成分股分析 - 了解指数构成和权重分布
   • 指数比较研究 - 对比不同指数的特征
   • 投资组合构建 - 基于指数成分股构建投资策略
   • 美股指数监控 - 跟踪美国市场主要指数表现
